# RNN

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
from data.generater import waveData

seq_len = 96  # 序列长度
in_dim = 16  # 输入维度
state_dim = 12  # 状态维度

xs, ys = waveData(1000, in_dim, seq_len)

train_xs = xs[:700]
train_ys = ys[:700]
test_xs = xs[700:]
test_ys = ys[700:]

print(train_xs.shape, train_ys.shape)
print(train_xs[0], train_ys[0])

(700, 96, 16) (700, 2)
[[ 0.70067267  1.09767075  1.71046556 ...  1.44020269  1.42304338
   1.24774039]
 [ 0.2938712   0.85894091  2.01797972 ...  0.75407059  0.93394378
   1.59527254]
 [ 1.38295959  1.71432963  1.10100853 ...  0.58260669  2.42027676
   2.33753907]
 ...
 [-0.11892027 -1.10122582 -1.38127633 ... -1.28338248  0.08651137
  -2.50751836]
 [-0.00366689 -0.90429202  0.08133322 ... -0.72773929 -0.63553461
  -0.37435678]
 [-0.28939092 -1.65019108 -1.23763818 ...  0.15221053 -1.90733885
  -0.41478961]] [0. 1.]


In [3]:
# 训练
import numpy as np
from mflow import core, ops, opts, lays

# 超参数
lr = 0.005
epoch = 10  # 30
batch_size = 16

with core.NameScope("RNN"):
    # 初始化变量
    inputs = [core.Variable(size=(in_dim, 1), trainable=False) for _ in range(seq_len)]
    u = core.Variable(size=(state_dim, in_dim), trainable=True)
    w = core.Variable(size=(state_dim, state_dim), trainable=True)
    b = core.Variable(size=(state_dim, 1), trainable=True)
    y = core.Variable(size=(2, 1), trainable=False)
    last_step = None  # 上一步输出，第一步的话就设置为None
    # 网络构建
    for iv in inputs:
        h = ops.Add(ops.MatMul(u, iv), b)
        if last_step is not None:
            h = ops.Add(ops.MatMul(w, last_step), h)
        h = ops.ReLU(h)
        last_step = h
    fc_1 = lays.Linear(last_step, state_dim, 40, "ReLU")
    fc_2 = lays.Linear(fc_1, 40, 10, "ReLU")
    pred = lays.Linear(fc_2, 10, 2, None)
    predicter = ops.Logistic(pred)
    loss = ops.loss.CrossEntropyWithSoftMax(pred, y)
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, lab) in enumerate(zip(train_xs, train_ys)):
            for j, x in enumerate(inputs):
                x.setValue(np.mat(feat[j]).T)
            y.setValue(np.mat(lab).T)
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                if (i + 1) % 64 == 0:
                    print("Epoch: {:d}, itet: {:d}, loss: {:.7f}.".format(
                        ep, i + 1, loss.value[0, 0]))
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in test_xs:
            for j, x in enumerate(inputs):
                x.setValue(np.mat(feat[j]).T)
            predicter.forward()
            preds.append(predicter.value.A.ravel())  # 结果
        preds = np.array(preds).argmax(axis=1)
        trues = test_ys.argmax(axis=1)
        acc = (trues == preds).astype("uint8").sum() / len(test_xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, itet: 64, loss: 0.7083669.
Epoch: 1, itet: 128, loss: 0.6639147.
Epoch: 1, itet: 192, loss: 0.6552973.
Epoch: 1, itet: 256, loss: 0.6511815.
Epoch: 1, itet: 320, loss: 0.7392856.
Epoch: 1, itet: 384, loss: 0.7383916.
Epoch: 1, itet: 448, loss: 0.6777211.
Epoch: 1, itet: 512, loss: 0.7561344.
Epoch: 1, itet: 576, loss: 1.0101479.
Epoch: 1, itet: 640, loss: 0.8131289.
Epoch: 1, acc: 0.893.
Epoch: 2, itet: 64, loss: 0.2841931.
Epoch: 2, itet: 128, loss: 0.1043078.
Epoch: 2, itet: 192, loss: 0.0004203.
Epoch: 2, itet: 256, loss: 0.0000745.
Epoch: 2, itet: 320, loss: 0.0000045.
Epoch: 2, itet: 384, loss: 0.0000002.
Epoch: 2, itet: 448, loss: -0.0000000.
Epoch: 2, itet: 512, loss: -0.0000000.
Epoch: 2, itet: 576, loss: -0.0000000.
Epoch: 2, itet: 640, loss: -0.0000000.
Epoch: 2, acc: 1.000.
Epoch: 3, itet: 64, loss: 0.0000013.
Epoch: 3, itet: 128, loss: -0.0000000.
Epoch: 3, itet: 192, loss: -0.0000000.
Epoch: 3, itet: 256, loss: -0.0000000.
Epoch: 3, itet: 320, loss: -0.0000000.
E